<a href="https://colab.research.google.com/github/NxrFesdac/bourbaki-nlp-avanzado/blob/main/modulo4/NED_unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import re, json, os
import spacy
from getpass import getpass as get_pass
from openai import OpenAI

# OpenAI Client
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = get_pass("Introduce tu OPENAI_API_KEY: ")
client = OpenAI()

ALLOWED = {"PERSON", "LOC", "ORG", "MISC"}

# Corpus
textos = [
    "En Santa Cecilia, Miguel admira a Ernesto de la Cruz en un póster del festival y sueña con tocar en la plaza.",
    "En la casa de los Rivera, Abuelita Elena insiste en que la música está prohibida, aunque Miguel practica la guitarra a escondidas.",
    "Durante el Día de Muertos, Miguel entra al mausoleo de Ernesto de la Cruz y toma su guitarra para el concurso.",
    "Al llegar a la Tierra de los Muertos, Miguel conoce a Héctor y a Dante cerca del puente de cempasúchil.",
    "Héctor busca su foto en la ofrenda para poder cruzar y ver a su familia en el mundo de los vivos.",
    "Miguel interpreta Recuérdame para Coco y ella recuerda a su papá.",
    "Imelda enfrenta a Ernesto de la Cruz en el escenario durante su gran fiesta.",
    "En el taller de los Rivera, la familia prepara la ofrenda con la foto de Mamá Imelda."
]

# Traer labels by text en un import para que el ejercicio no este viciado y así se puedan escoger entidades sin saber cual es la KB.
labels_by_text = {
    0: {"santa cecilia":"LOC", "miguel":"PERSON", "ernesto de la cruz":"PERSON"},
    1: {"rivera":"ORG", "abuelita elena":"PERSON", "miguel":"PERSON"},
    2: {"día de muertos":"MISC", "miguel":"PERSON", "ernesto de la cruz":"PERSON"},
    3: {"tierra de los muertos":"LOC", "miguel":"PERSON", "héctor":"PERSON", "dante":"MISC", "puente de cempasúchil":"LOC"},
    4: {"héctor":"PERSON"},
    5: {"miguel":"PERSON", "recuérdame":"MISC", "coco":"PERSON"},
    6: {"imelda":"PERSON", "ernesto de la cruz":"PERSON"},
    7: {"rivera":"ORG", "mamá imelda":"PERSON"},
}

i = 4  # Índice del texto a probar
text = textos[i]
gold_dict = labels_by_text[i]
gold_mentions = {m.lower() for m in gold_dict.keys()}
gold_full = {(m.lower(), t) for m, t in gold_dict.items()}

print(f"\n--- TRABAJANDO CON EL TEXTO {i} ---")
print(f"TEXTO: {text}\n")

def calcular_metricas(gold, pred, nombre):
    tp = gold & pred
    fp = pred - gold
    fn = gold - pred
    print(f"\nResultados para [{nombre}]:")
    print(f"  - TP (Aciertos): {len(tp)} | FP (Sobran): {len(fp)} | FN (Faltan): {len(fn)}")
    print(f"  - Detalles TP: {sorted(tp)}")
    if fp: print(f"  - Detalles FP: {sorted(fp)}")
    if fn: print(f"  - Detalles FN: {sorted(fn)}")

Introduce tu OPENAI_API_KEY: ··········

--- TRABAJANDO CON EL TEXTO 4 ---
TEXTO: Héctor busca su foto en la ofrenda para poder cruzar y ver a su familia en el mundo de los vivos.



In [ ]:
# Ejercicio 1, extraer entidades a mano
print("--- EJERCICIO 1: EXTRAER SOLO MENCIONES ---")
user_input_1 = input("Tú: Escribe las entidades separadas por coma: ")
user_pred_1 = {m.strip().lower() for m in user_input_1.split(",") if m.strip()}

# ChatGPT
prompt_ej1 = f"Extrae entidades del siguiente texto {text} \n por favor retornalos como una lista ([]), no escribas nada más, solo retorna la lista"
r1 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":prompt_ej1}], temperature=0)
gpt_pred_1 = {m.lower() for m in json.loads(r1.choices[0].message.content)}

calcular_metricas(gold_mentions, user_pred_1, "USUARIO (Menciones)")
calcular_metricas(gold_mentions, gpt_pred_1, "CHATGPT (Menciones)")

--- EJERCICIO 1: EXTRAER SOLO MENCIONES ---
Tú: Escribe las entidades separadas por coma: héctor

Resultados para [USUARIO (Menciones)]:
  - TP (Aciertos): 1 | FP (Sobran): 0 | FN (Faltan): 0
  - Detalles TP: ['héctor']

Resultados para [CHATGPT (Menciones)]:
  - TP (Aciertos): 1 | FP (Sobran): 4 | FN (Faltan): 0
  - Detalles TP: ['héctor']
  - Detalles FP: ['familia', 'foto', 'mundo de los vivos', 'ofrenda']


In [ ]:
# Ejercicio 2, menciones a mano con categoria
print("Ejemplo de formato: Miguel:PERSON, Santa Cecilia:LOC")
user_input_2 = input("Tú: Escribe 'Mención:Tipo' separadas por coma: ")
user_pred_2 = set()
for item in user_input_2.split(","):
    if ":" in item:
        m, t = item.split(":")
        user_pred_2.add((m.strip().lower(), t.strip().upper()))

# ChatGPT
prompt_ej2 = f"Extrae entidades y tipos (PERSON, LOC, ORG, MISC) como diccionario con la siguiente estructura: {{\"entities\": [{{\"mention\":\"\", \"type\":\"\"}}]}}: {text}, por favor solo retorna el diccionario y nada más"
r2 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":prompt_ej2}], temperature=0)
gpt_raw_2 = json.loads(r2.choices[0].message.content)["entities"]
gpt_pred_2 = {(e["mention"].lower(), e["type"].upper()) for e in gpt_raw_2}

calcular_metricas(gold_full, user_pred_2, "USUARIO (Mención + Tipo)")
calcular_metricas(gold_full, gpt_pred_2, "CHATGPT (Mención + Tipo)")

Ejemplo de formato: Miguel:PERSON, Santa Cecilia:LOC
Tú: Escribe 'Mención:Tipo' separadas por coma: héctor:PERSON

Resultados para [USUARIO (Mención + Tipo)]:
  - TP (Aciertos): 1 | FP (Sobran): 0 | FN (Faltan): 0
  - Detalles TP: [('héctor', 'PERSON')]

Resultados para [CHATGPT (Mención + Tipo)]:
  - TP (Aciertos): 1 | FP (Sobran): 0 | FN (Faltan): 0
  - Detalles TP: [('héctor', 'PERSON')]


In [ ]:
# Ejercicio 3, utilizar NER
nlp = spacy.load("es_core_news_lg")
doc = nlp(text)
SPACY2TOY = {"PER":"PERSON", "LOC":"LOC", "ORG":"ORG", "MISC":"MISC"}
spacy_pred = {(e.text.lower(), SPACY2TOY.get(e.label_, "MISC")) for e in doc.ents}

calcular_metricas(gold_full, spacy_pred, "SPACY (Mención + Tipo)")


Resultados para [SPACY (Mención + Tipo)]:
  - TP (Aciertos): 1 | FP (Sobran): 0 | FN (Faltan): 0
  - Detalles TP: [('héctor', 'PERSON')]
